# 线性回归模型

MXNet 实现简单的线性回归模型，采用 Gluon 接口，更方便简洁。

In [5]:
from mxnet import autograd, nd
from mxnet.gluon import data as gdata

## 生成数据集

In [6]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

## 读取数据

In [7]:
batch_size = 10

# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
# 随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

## 定义模型

In [8]:
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(1)) # 添加一个输出个数为 1 的全连接层

## 初始化模型参数

我们通过 init.Normal(sigma=0.01) 指定权重参数每个元素将在初始化时随机采样于均值为 0、标准差为 0.01 的正态分布。偏差参数默认会初始化为零。

In [9]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

## 定义损失函数

In [10]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()  # 平方损失又称 L2 范数损失

## 定义优化算法

In [11]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

## 训练模型

In [12]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.054451
epoch 2, loss: 0.000247
epoch 3, loss: 0.000050
epoch 4, loss: 0.000049
epoch 5, loss: 0.000049
epoch 6, loss: 0.000049
epoch 7, loss: 0.000049
epoch 8, loss: 0.000049
epoch 9, loss: 0.000049
epoch 10, loss: 0.000049


比较一下学到的参数和用来生成训练集的真实参数。

In [13]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 2.0001836 -3.3995354]]
 <NDArray 1x2 @cpu(0)>)

In [14]:
true_b, dense.bias.data()

(4.2, 
 [4.199757]
 <NDArray 1 @cpu(0)>)